In [ ]:
!pip install confluent_kafka sseclient

In [1]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import sys, random

brokers = "kafka1:9092,kafka2:9093"

In [2]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer':  StringSerializer('utf_8')
}

In [3]:
p = SerializingProducer(pconf)

In [4]:
stocks = "local/stocks.csv"

In [5]:
from datetime import datetime  
  #2023-10-13T08:16:13Z
def construct_stock(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ") #"%d-%m-%Y, %H:%M:%S"
    stock = {"name": row[6],
             "price": float(row[2]),
             "timestamp":str_date_time
             }
    return stock

In [ ]:
import csv, json
import time
n = 0
with open(stocks) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    try:
        for row in csv_reader:
            #if n == 100:
             #  break
            stock = construct_stock(row)
            print(stock)
            p.produce('stock', value=json.dumps(stock))
            p.poll(0)
            p.flush()
            time.sleep(0.5)
            #n = n + 1
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

    
        
    
    
    

{'name': 'ISRG', 'price': 192.6465, 'timestamp': '2024-12-03T18:18:32Z'}
{'name': 'BBT', 'price': 30.8, 'timestamp': '2024-12-03T18:18:33Z'}
{'name': 'FITB', 'price': 16.63, 'timestamp': '2024-12-03T18:18:33Z'}
{'name': 'ZION', 'price': 24.57, 'timestamp': '2024-12-03T18:18:34Z'}
{'name': 'HCP', 'price': 47.08, 'timestamp': '2024-12-03T18:18:34Z'}
{'name': 'LLY', 'price': 53.91, 'timestamp': '2024-12-03T18:18:35Z'}
{'name': 'MAC', 'price': 62.01, 'timestamp': '2024-12-03T18:18:35Z'}
{'name': 'ADBE', 'price': 39.15, 'timestamp': '2024-12-03T18:18:36Z'}
{'name': 'AZO', 'price': 384.01, 'timestamp': '2024-12-03T18:18:36Z'}
{'name': 'CL', 'price': 54.165, 'timestamp': '2024-12-03T18:18:37Z'}
{'name': 'PPG', 'price': 69.885, 'timestamp': '2024-12-03T18:18:37Z'}
{'name': 'KMB', 'price': 90.08, 'timestamp': '2024-12-03T18:18:38Z'}
{'name': 'AMAT', 'price': 13.99, 'timestamp': '2024-12-03T18:18:38Z'}
{'name': 'KR', 'price': 14.205, 'timestamp': '2024-12-03T18:18:39Z'}
{'name': 'WEC', 'price': 

### Use only if you need to purge all the messages in the queue

In [ ]:
brokers = "kafka1:9092,kafka2:9093"

admin_client = AdminClient({"bootstrap.servers":brokers})
admin_client.delete_topics(topics=['stock'])